In [1]:
import numpy as np
import scipy.io as sio
from numpy import linalg as LA
import math
import sounddevice as sd

In [2]:
#Blanchisement of signal
dataSignal = sio.loadmat("SignauxMelange.mat")
x1 = np.array(dataSignal['v_melange_out_1']).reshape(1,88200)
x2 = np.array(dataSignal['v_melange_out_2']).reshape(1,88200)
n= x1.shape[1]
X = np.concatenate((x1, x2), axis=0)
w, v = LA.eig(X.dot(X.T))
D = np.diag(w**(-1/2))
xTilde = np.dot(v, D).dot(v.T).dot(X)

#Choose randomly the value of w1 and w2 
w1 = np.array([[0.7],[-0.6]]) # (0.5,0.2) pire   (0.7, 0.8)
w2 = np.array([[0.35],[0.21]]) 

In [3]:
def Firstcompo(w):
    #while(LA.norm(w-w_1)>0.1): # colinear condition de convergence est different
    for i in range(1000):
        gPrime = np.sum(1.- np.tanh(w.T.dot(xTilde))**2)/n 
        g = xTilde * np.tanh(w.T.dot(xTilde))
        g = np.sum(g, axis=1,keepdims=True)/n
        wPlus = g - gPrime*w
        w=wPlus/LA.norm(wPlus)
    return w

In [4]:
#Load reference signals
dataReference = sio.loadmat("SignauxReference.mat")    
s1 =  np.array(dataReference['v_melange_in_1']).reshape(1,88200)
s2 =  np.array(dataReference['v_melange_in_2']).reshape(1,88200)

In [5]:
def EQMN(s, sHat):
    eqmn=1-(float(s.dot(sHat.T))/(LA.norm(s)*LA.norm(sHat)))**2
    return 10*math.log(eqmn)

In [6]:
#Get the first w
w1=Firstcompo(w1)


# Implement orthogonalization of Gram-Schmidt  
w2=w2-float(w1.T.dot(w2))*w1
w2=w2/LA.norm(w2)


y1 = w1.T.dot(xTilde)
eqmn_1_w1 = EQMN(y1, s1)
eqmn_2_w1 = EQMN(y1, s2)
print("EQMN:")
print(eqmn_1_w1)

y2 = w2.T.dot(xTilde)
eqmn_1_w2 = EQMN(y2, s1)
eqmn_2_w2 = EQMN(y2, s2)
print("EQMN:")
print(eqmn_2_w2)


EQMN:
-55.53635209994206
EQMN:
-55.28980563482729


In [7]:
print("The mixed signal 1.")
sd.play(x1.reshape(88200,),44100)

The mixed signal 1.


In [8]:
print("The mixed signal 2.")
sd.play(x2.reshape(88200,),44100)

The mixed signal 2.


In [9]:
print("The reference signal 1.")
sd.play(s1.reshape(88200,),44100)

The reference signal 1.


In [10]:
print("The reference signal 2.")
sd.play(s2.reshape(88200,),44100)

The reference signal 2.


In [11]:
print("The seperated signal 1.")
sd.play(y1.reshape(88200,),44100)

The seperated signal 1.


In [12]:
print("The seperated signal 2.")
sd.play(y2.reshape(88200,),44100)

The seperated signal 2.
